In [ ]:
%pip install pandas 
%pip install numpy 
%pip install spacy
%pip install -U scikit-learn
%pip install seaborn
%pip install transformers
# %pip install fasttext
%pip install lazypredict
%pip install gensim

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
import fasttext
import fasttext.util
from lazypredict.Supervised import LazyClassifier
from transformers import BertTokenizer
import gensim
import gensim.downloader

2023-07-07 19:36:35.857022: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 19:36:36.869391: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
davidson = pd.read_csv("datasets/edited/davidson_sentiment_score.csv")
davidson

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,care.virtue,care.vice,fairness.virtue,...,loyalty.virtue,loyalty.vice,authority.virtue,authority.vice,sanctity.virtue,sanctity.vice,cleaned_tweets,positive,neutral,negative
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,0,0,0,...,0,0,0,0,1,1,rt mayasolovely woman complain clean house am...,0.07,0.41,0.52
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,0,0,...,0,0,0,0,0,0,rt mleew boy dat cold tyga dwn bad cuffin d...,0.03,0.43,0.54
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,0,0,...,0,0,0,0,0,2,rt urkindofbrand dawg rt sbaby life fuck b...,0.00,0.03,0.97
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,0,0,...,0,0,0,0,0,0,rt c g anderson viva based look like tranny,0.06,0.73,0.21
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,0,0,...,0,0,0,0,0,1,rt shenikaroberts shit hear true faker bitch ...,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0,0,0,...,0,0,0,0,1,1,muthaf in lie lifeasking pearls c...,0.01,0.12,0.87
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an...",0,0,0,...,0,0,0,0,0,0,go break wrong heart baby drive redneck crazy,0.01,0.10,0.89
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...,0,0,0,...,0,0,0,0,0,0,young buck wanna eat dat nigguh like be nt fuc...,0.04,0.39,0.57
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies,0,0,0,...,0,0,0,0,0,0,youu got wild bitch tellin lie,0.01,0.11,0.88


In [3]:
from pathlib import Path

def getTrainSetFastText():
    ft_model = fasttext.load_model(str(Path("models/crawl-300d-2M-subword.bin")))
    fasttext.util.reduce_model(ft_model, 100)
    traindata = []
    for tweet in davidson['tweet']:
        traindata.append(ft_model.get_sentence_vector(tweet.replace("\n", "")))
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    traindata["positive"] = davidson["positive"].values.tolist()
    traindata["neutral"] = davidson["neutral"].values.tolist() 
    traindata["negative"] = davidson["negative"].values.tolist() 
    return traindata

def getTrainSetTFIDF():
    countvec = CountVectorizer(max_features=100)
    bow = countvec.fit_transform(davidson['tweet']).toarray()
    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(bow).toarray()
    training_data = pd.DataFrame(X)
    training_data.columns = training_data.columns.astype(str)
    return training_data

def getTrainSetBERT():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_data = tokenizer(davidson['tweet'].tolist(), padding=True, truncation=True, max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetWord2Vec():
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format('word2vec-google-news-300.bin', binary=True)
    traindata = []
    for msg in davidson['tweet']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in w2v_model:
                vecs.append(w2v_model[word][:100])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetGlove():
    glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')
    traindata = []
    for msg in davidson['tweet']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in glove_vectors:
                vecs.append(glove_vectors[word])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

In [4]:
X = getTrainSetFastText()
X